In [10]:
import pandas as pd

df = pd.read_csv('cleaned.csv')

df

,Upvotes,Date,Title,Body Text,Link
0,10,1449121338,Breaking Through the Safeties,Finally heard back from all of my safeties! O...,https://www.reddit.com/r/collegeresults/commen...
1,9,1449874637,Virginia Tech ED Deferred :(,Demographics: In-state Asian male\n\nIntended ...,https://www.reddit.com/r/collegeresults/commen...
2,6,1449888267,Accepted To Middlebury ED,Middlebury ED (only application)\n\nno financi...,https://www.reddit.com/r/collegeresults/commen...
3,6,1449900824,Accepted to Vanderbilt School of Engineering ED,Schools: Vanderbilt University (Accepted ED)\n...,https://www.reddit.com/r/collegeresults/commen...
4,7,1449936672,Accepted into Duke Pratt ED,I was just accepted this past Thursday to my f...,https://www.reddit.com/r/collegeresults/commen...
...,...,...,...,...,...
5598,1,1720456114,This Sub as issues...,Why is it that a black//Latine person will sha...,https://www.reddit.com/r/collegeresults/commen...
5599,1,1720471080,EU student bags ivies and top schools,**Demographics**\n\n* Gender: Male \n* Race/Et...,https://www.reddit.com/r/collegeresults/commen...
5600,1,1720484984,"Great Supplementals, Got into T20s",**Demographics**\n\n* Gender: Female\n* Race/E...,https://www.reddit.com/r/collegeresults/commen...
5601,1,1720485521,"Psych Major, Got into T20s, Great Essays",**Demographics**\n\n* Gender: Female\n* Race/E...,https://www.reddit.com/r/collegeresults/commen...


In [11]:
df2 = df
df2.drop(['Upvotes', 'Date', 'Title'], axis=1, inplace=True)


df2 = df2.tail(10) #only lsat 10 posts

df2


,Body Text,Link
5593,**Demographics**\n\n* Gender: Male\n* Race/Eth...,https://www.reddit.com/r/collegeresults/commen...
5594,"Hey y'all, first time posting just wanted to k...",https://www.reddit.com/r/collegeresults/commen...
5595,**Demographics**\n\n* Gender: Male\n* Race/Eth...,https://www.reddit.com/r/collegeresults/commen...
5596,I am an Asian from a top bay area high school....,https://www.reddit.com/r/collegeresults/commen...
5597,**Demographics**\n\n* Female \n* Chinese :D \n...,https://www.reddit.com/r/collegeresults/commen...
5598,Why is it that a black//Latine person will sha...,https://www.reddit.com/r/collegeresults/commen...
5599,**Demographics**\n\n* Gender: Male \n* Race/Et...,https://www.reddit.com/r/collegeresults/commen...
5600,**Demographics**\n\n* Gender: Female\n* Race/E...,https://www.reddit.com/r/collegeresults/commen...
5601,**Demographics**\n\n* Gender: Female\n* Race/E...,https://www.reddit.com/r/collegeresults/commen...
5602,People always seem to mention race but never g...,https://www.reddit.com/r/collegeresults/commen...


In [15]:
import re
import pandas as pd

def extractData(row, pattern, matchGroup=1):
    text = str(row['Body Text'])
    pattern = re.compile(pattern, re.IGNORECASE)
    match = pattern.search(text)
    
    if match:
        return match.group(matchGroup)  # Don't include the search part before it, ONLY the captured GROUP!
    else:
        return -1


def extractWithIndex(row, start, end):
    #prep
    text = str(row['Body Text'])
    text = text.lower()
    start = start.lower()

    start = text.find(start)


    #end is an array of potential ends. we want the first index that is not -1
    actualMinEnd = -1
    for i in end:
        i = i.lower()
        end = text.find(i)
        if end != -1:
            actualMinEnd = end
            break
    
            
    if start != -1 and actualMinEnd != -1:
        return text[start:actualMinEnd]
    else:
        return -1
    


new_df = pd.DataFrame()

new_df['Body Text'] = df2['Body Text']
new_df['Link'] = df2['Link']

#DEMOGRAPHICS
new_df['Gender'] = df2.apply(lambda row: extractData(row, r"Gender:\s*(.*)"), axis=1)
new_df['Race'] = df2.apply(lambda row: extractData(row, r"Race\/Ethnicity:\s*(.*)"), axis=1)
new_df['Residence'] = df2.apply(lambda row: extractData(row, r"Residence:\s*(.*)"), axis=1)
new_df['Income'] = df2.apply(lambda row: extractData(row, r"Income Bracket:\s*(.*)"), axis=1)
new_df['School Type'] = df2.apply(lambda row: extractData(row, r"Type of School:\s*(.*)"), axis=1)
new_df['Hooks'] = df2.apply(lambda row: extractData(row, r"(?:Hooks \(Recruited Athlete, URM, First-Gen, Geographic, Legacy, etc.\)|Hooks):\s*(.*)"), axis=1)

#ACADEMICS
new_df['Major'] = df2.apply(lambda row: extractData(row, r"Intended Major\(s\)\**:*(.*)"), axis=1)
new_df['GPA'] = df2.apply(lambda row: extractData(row, r"GPA \(UW\/W\)\**:*(.*)"), axis=1)
new_df['Rank'] = df2.apply(lambda row: extractData(row, r"Rank \(or percentile\):(.*)"), axis=1)
new_df['APs'] = df2.apply(lambda row: extractData(row, r".*Enrollment\/etc:(.*)"), axis=1)
new_df['SeniorCourseLoad'] = df2.apply(lambda row: extractData(row, r".*Course Load:(.*)"), axis=1)


#testing
new_df['SAT'] = df2.apply(lambda row: extractData(row, r"SAT.*:(.*)"), axis=1)
new_df['ACT'] = df2.apply(lambda row: extractData(row, r"ACT.*:(.*)"), axis=1)

#activities
new_df['EC'] = df2.apply(lambda row: extractWithIndex(row, "Extracurriculars/Activities", ["Awards/Honor", "Letters of Recommendation", "Interviews", "Essays", "Decisions", "Acceptances" ]), axis=1)
new_df['Awards'] = df2.apply(lambda row: extractWithIndex(row, "Awards/Honor", ["Letters of Recommendation", "Interviews", "Essays", "Decisions", "Acceptances"] ), axis=1)
new_df['LOR'] = df2.apply(lambda row: extractWithIndex(row, "Letters of Recommendation", [ "Interviews", "Essays","Decisions", "Acceptances"]), axis=1)
new_df['Interviews'] = df2.apply(lambda row: extractWithIndex(row, "Interviews", ["Essays", "Decisions", "Acceptances"]), axis=1)
new_df['Essays'] = df2.apply(lambda row: extractWithIndex(row, "Essays", ["Decisions", "Acceptances"]), axis=1)


new_df




,Body Text,Link,Gender,Race,Residence,Income,School Type,Hooks,Major,GPA,Rank,APs,SeniorCourseLoad,SAT,ACT,EC,Awards,LOR,Interviews,Essays
5593,**Demographics**\n\n* Gender: Male\n* Race/Eth...,https://www.reddit.com/r/collegeresults/commen...,Male,Black (Togolese/Nigerian),MA,80K-100K,"non-competitive small private school, no one h...","urm, lgbtq+, fgli","sociology, public health-like majors, biology...","3.99 UW, 4.55 W",1/55,-1,"5 APs, 2 Honors (took seven classes instead o...",-1,-1,extracurriculars/activities**\n\n*list all ext...,awards/honors**\n\n*list all awards and honors...,,-1,"essays, prose pieces\n\n**letters of recommend..."
5594,"Hey y'all, first time posting just wanted to k...",https://www.reddit.com/r/collegeresults/commen...,Male,Mexican,WI,< 70K for family of 7 including me,Fairly Competitive Public School in my State (...,"First-gen, low-income",Computer Science,"3.49 UW, 3.94 W",38th out of 98 students,-1,4 Computer Science Courses (spread over 4 qua...,-1,-1,extracurriculars/activities**\n\n*list all ext...,awards/honors**\n\n*list all awards and honors...,letters of recommendation**\n\n1. i forgot to ...,-1,essays**\n\ni wrote about how my interest in c...
5595,**Demographics**\n\n* Gender: Male\n* Race/Eth...,https://www.reddit.com/r/collegeresults/commen...,Male,East Asian,FL,Upper Class,competitive public,None,Computer Science,"4.0 UW, 4.5 W",16/300,-1,5 APs,1560,-1,extracurriculars/activities**\n\n*list all ext...,awards/honors**\n\n*list all awards and honors...,letters of recommendation**\n\n1. physics teac...,-1,essays**\n\nwrote my personal statement on som...
5596,I am an Asian from a top bay area high school....,https://www.reddit.com/r/collegeresults/commen...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5597,**Demographics**\n\n* Female \n* Chinese :D \n...,https://www.reddit.com/r/collegeresults/commen...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,interviews - 3 yrs\n* museum guide - 2 yrs\n* ...,-1
5598,Why is it that a black//Latine person will sha...,https://www.reddit.com/r/collegeresults/commen...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5599,**Demographics**\n\n* Gender: Male \n* Race/Et...,https://www.reddit.com/r/collegeresults/commen...,Male,White,European Union,\~$120k,-1,Strong Math profile + Medalist,Mathematics and/ or Computer Science,IB 45/45 (actual) 43/45(Predicted),1/83,-1,-1,(,-1,extracurriculars/activities**\n\n1. national m...,awards/honors**\n\n1. imo silver medal \n2. 2x...,letters of recommendation**\n\nmajor politicia...,-1,essays**\n\npersonal essay (10/10): i wrote an...
5600,**Demographics**\n\n* Gender: Female\n* Race/E...,https://www.reddit.com/r/collegeresults/commen...,Female,Black (African American),IL,50k-75k,Private Catholic,N/A,Psychology,3.98 W,N/A,-1,"2 APs, 3H",-1,-1,extracurriculars/activities**\n\n*list all ext...,awards/honors**\n\n*list all awards and honors...,letters of recommendation**\n\n(*briefly descr...,"interviews**\n\ninterviewed at harvard, did no...","essays**\n\ni wrote about a serious, life-thre..."
5601,**Demographics**\n\n* Gender: Female\n* Race/E...,https://www.reddit.com/r/collegeresults/commen...,Female,Black (African American),IL,50k-75k,Private Catholic,"URM, Took Latin and Ancient Greek","Psychology, Classics (as second choice so I w...",3.98 W,N/A,-1,"AP Psych & AP Latin, 3H",-1,-1,extracurriculars/activities**\n\n*list all ext...,awards/honors**\n\n*list all awards and honors...,letters of recommendation**\n\n(*briefly descr...,"interviews**\n\ninterviewed at harvard, did no...","essays**\n\ni wrote about a serious, life-thre..."
5602,People always seem to mention race but never g...,https://www.reddit.com/r/collegeresults/commen...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
